In [1]:
import numpy as np
from ipyleaflet import Map, basemaps, basemap_to_tiles, CircleMarker, Polyline, AntPath
from ipywidgets import HTML
import pickle
from dijkstar import Graph, find_path

In [2]:
start_city = None
delivery_city = []
cities_chosen = []

In [3]:
def convert_min_hour(nb_min):
  nb_min += 8 * 60
  nb_hour = nb_min // 60
  nb_min = nb_min % 60
  return str(nb_hour) + "h" + str(nb_min) + "min"

def time_travel(graph, path):
  time_travel = 0
  time = {}
  for i in range(len(path) - 1):
    if path[i] not in time.keys():
      time[path[i]] = [time_travel]
    else:
      time[path[i]].append(time_travel)
    time_travel += graph[path[i]][path[i+1]]
  time[path[i + 1]].append(time_travel)
  return time

def get_message(city, total_time):
  output = ""
  output += city['name']
  for i, heures in enumerate(total_time[int(city["id"])]):
    output += "<br> Passage " + str(i + 1) + " : " + convert_min_hour(heures)
  return output

def draw_cities(m, time = False):
  if time:
    total_time = time_travel(graph, complete_path)
  for city in cities:
    popupMessage = HTML()
    circle_marker = CircleMarker()
    circle_marker.location = city['coords']
    circle_marker.radius = 10
    circle_marker.fill_opacity = 1
    if city["name"] == start_city:
      circle_marker.color = "red"
      circle_marker.fill_color = "red"
      popupMessage.value = city['name']
      if time :
        popupMessage.value = get_message(city, total_time)
    elif city['name'] in delivery_city:
      circle_marker.color = "orange"
      circle_marker.fill_color = "orange"
      popupMessage.value = city['name']
      if time :
        popupMessage.value = get_message(city, total_time)
    elif city['name'] in cities_chosen:
      circle_marker.color = "green"
      circle_marker.fill_color = "green"
      popupMessage.value = city['name']
      if time :
        popupMessage.value = get_message(city, total_time)
    else:
      circle_marker.color = "gray"
      circle_marker.fill_color = "gray"
      popupMessage.value = city['name']
    circle_marker.popup = popupMessage
    m.add_layer(circle_marker)

def draw_path(m, path):
  path_list = []
  for i in range(len(path) - 1):
    path_list.append([cities[path[i]]['coords'], cities[path[i+1]]['coords']])
  ant_path = AntPath(
    locations=path_list,
    dash_array=[1, 10],
    delay=1000,
    color='red',
    pulse_color='pink'
  )
  m.add_layer(ant_path)

def draw_lines(m, graph):
  all_paths = []
  for i in range(len(graph)):
    for j in range(len(graph)):
      if graph[i][j] != np.inf:
        all_paths.append([cities[i]['coords'], cities[j]['coords']])
  line = Polyline(
      locations=all_paths,
      color="gray",
      fill=False
  )
  m.add_layer(line)

def convert_graph(graph):
  graph_dijkstra = Graph()
  for i in range(len(graph)):
    for j in range(len(graph)):
      graph_dijkstra.add_edge(i, j, graph[i][j])
  return graph_dijkstra

def get_distance_between_vertices(graph, i, j):
  if (graph[i][j] != np.inf):
    return graph[i][j], [i, j]
  else:
    if i != j:
      shortest_path = find_path(graph, i,j)
      return shortest_path.total_cost, shortest_path.nodes
    else:
      return 0, None

def get_complete_graph(graph, vertices_list):
  all_paths = {}
  converted_graph = convert_graph(graph)
  complete_graph = np.zeros((len(vertices_list), len(vertices_list)))
  for vertex in vertices_list:
    successors = np.delete(vertices_list, np.where(vertices_list == vertex))
    for successor in successors:
      distance, path = get_distance_between_vertices(converted_graph, vertex, successor)
      i = np.where(np.array(vertices_list) == vertex)[0][0]
      j = np.where(np.array(vertices_list) == successor)[0][0]
      complete_graph[i][j] = distance
      all_paths[(i, j)] = path
  return complete_graph, all_paths

def get_interval(hour_start, hour_end):
  return ((hour_start - 8) * 60, (hour_end - 8) * 60)

In [4]:
cities = [
  {
    "id": 0,
    "name": "Bezannes",
    "coords": (49.22363900671118, 3.9887723157745576)
  },
  {
    "id" : 1,
    "name": "Bétheny",
    "coords": (49.285131827708035, 4.063798713400151)
  },
  {
    "id" : 2,
    "name": "Cernay-les-Reims",
    "coords": (49.2642453710596, 4.1036913180272)
  },
  {
    "id" : 3,
    "name": "Champfleury",
    "coords": (49.19818615812236, 4.019795395571029)
  },
  {
    "id" : 4,
    "name": "Cormontreuil",
    "coords": (49.22264097621803, 4.049551196446118)
  },
  {
    "id" : 5,
    "name": "Cruny",
    "coords": (49.257059457427616, 3.7369142978612735)
  },
  {
    "id" : 6,
    "name": "Epoye",
    "coords": (49.29041023356542, 4.237901634457861)
  },
  {
    "id" : 7,
    "name": "Jonchery-sur-vesle",
    "coords": (49.28752839149448, 3.8121724734582907)
  },
  {
    "id" : 8,
    "name": "Muizon",
    "coords": (49.27682283902432, 3.889787835151326)
  },
  {
    "id" : 9,
    "name": "Nogent-l'Abbesse",
    "coords": (49.25586664716424, 4.155845017046866)
  },
  {
    "id" : 10,
    "name": "Poilly",
    "coords": (49.216141770809536, 3.8203569577722494)
  },
  {
    "id" : 11,
    "name": "Reims",
    "coords": (49.258352984690276, 4.030731630624372)
  },
  {
    "id" : 12,
    "name": "Taissy",
    "coords": (49.21423458122659, 4.093274461155123)
  },
  {
    "id" : 13,
    "name": "Tinqueux",
    "coords": (49.24715898263178, 3.9988975225936736)
  },
  {
    "id" : 14,
    "name": "Witry-les-Reims",
    "coords": (49.2933840208818, 4.125741773971905)
  }
]

In [5]:
graph = [
  [ np.inf, np.inf, np.inf, 12, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, 17, np.inf, np.inf, 7, np.inf],
  [ np.inf, np.inf, 9, np.inf, np.inf, np.inf, np.inf, np.inf, 23, np.inf, np.inf, 17, np.inf, np.inf, 8],
  [ np.inf, 9, np.inf, np.inf, 12, np.inf, np.inf, np.inf, np.inf, 7, np.inf, 15, np.inf, np.inf, 7],
  [12, np.inf, np.inf, np.inf, 12, np.inf, np.inf, np.inf, np.inf, np.inf, 21, np.inf, 16, np.inf, np.inf],
  [ np.inf, np.inf, 12, 12, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, 15, 9, 14, np.inf],
  [ np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, 13, np.inf, np.inf, 12, np.inf, np.inf, 28, np.inf],
  [ np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, 8, np.inf, np.inf, np.inf, np.inf, 10],
  [ np.inf, np.inf, np.inf, np.inf, np.inf, 13, np.inf, np.inf, 10, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf],
  [ np.inf, 23, np.inf, np.inf, np.inf, np.inf, np.inf, 10, np.inf, np.inf, np.inf, 19, np.inf, 13, np.inf],
  [ np.inf, np.inf, 7, np.inf, np.inf, np.inf, 8, np.inf, np.inf, np.inf, np.inf, np.inf, 10, np.inf, np.inf],
  [17, np.inf, np.inf, 21, np.inf, 12, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf],
  [ np.inf, 17, 15, np.inf, 15, np.inf, np.inf, np.inf, 19, np.inf, np.inf, np.inf, np.inf, 13, np.inf],
  [ np.inf, np.inf, np.inf, 16, 9, np.inf, np.inf, np.inf, np.inf, 10, np.inf, np.inf, np.inf, np.inf, np.inf],
  [ 7, np.inf, np.inf, np.inf, 14, 28, np.inf, np.inf, 13, np.inf, np.inf, 13, np.inf, np.inf, np.inf],
  [ np.inf, 8, 7, np.inf, np.inf, np.inf, 10, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf]
]

In [6]:
m_1 = Map(
  basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
  center=cities[11]['coords'],
  zoom=10.5,
)
draw_cities(m_1)

m_1

Map(center=[49.258352984690276, 4.030731630624372], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [7]:
m_1 = Map(
  basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
  center=cities[11]['coords'],
  zoom=10.5,
)

draw_lines(m_1, graph)
draw_cities(m_1)

m_1

Map(center=[49.258352984690276, 4.030731630624372], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [8]:
start_city = "Reims"

cities_chosen = ["Bétheny", "Cruny", "Epoye", "Jonchery-sur-vesle", "Champfleury", "Cormontreuil", "Taissy", "Cernay-les-Reims", "Witry-les-Reims"]

In [9]:
m_1 = Map(
  basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
  center=cities[11]['coords'],
  zoom=10.5,
)

draw_lines(m_1, graph)
draw_cities(m_1)

m_1

Map(center=[49.258352984690276, 4.030731630624372], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [10]:
cities_chosen.append(start_city)
cities_chosen_index = [cities.index(next(city for city in cities if city['name'] == city_choose)) for city_choose in cities_chosen]

complete_graph, all_paths = get_complete_graph(graph, cities_chosen_index)

In [11]:
start_vertex = cities.index(next(city for city in cities if city['name'] == start_city))
start_vertex_index = np.where(np.array(cities_chosen) == start_city)[0][0]

data = {
  "graph": graph,
  "complete_graph": complete_graph,
  "all_paths": all_paths,
  "vertices_list": cities_chosen,
  "start_vertex": start_vertex,
  "start_vertex_index": start_vertex_index,
  "interval_vertex": [12],
  "interval_vertex_index": [6],
  "interval": [(0,1000000000)],
}

with open('soutenance.pickle', 'wb') as file:
    pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
with open("soutenance_path.pickle", "rb") as file:
  data = pickle.load(file)

complete_path = data["complete_path"]
weight = data["weight"]
print(weight)

156.0


In [13]:
m_1 = Map(
  basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
  center=cities[11]['coords'],
  zoom=10.5,
)

draw_lines(m_1, graph)
draw_cities(m_1, True)
draw_path(m_1, complete_path)

m_1

Map(center=[49.258352984690276, 4.030731630624372], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [14]:
delivery_interval = [get_interval(8, 9), get_interval(9, 10),get_interval(11, 12)]
delivery_city = ["Cormontreuil","Bétheny", "Jonchery-sur-vesle"]

In [15]:
interval_vertex = [cities.index(next(city for city in cities if city['name'] == delivery_cit)) for delivery_cit in delivery_city]
interval_vertex_index = [np.where(np.array(cities_chosen) == delivery_cityy)[0][0] for delivery_cityy in delivery_city]

data = {
  "graph": graph,
  "complete_graph": complete_graph,
  "all_paths": all_paths,
  "vertices_list": cities_chosen,
  "start_vertex": start_vertex,
  "start_vertex_index": start_vertex_index,
  "interval_vertex": interval_vertex,
  "interval_vertex_index": interval_vertex_index,
  "interval": delivery_interval,
}

with open('soutenance_interval.pickle', 'wb') as file:
    pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
with open("soutenance_interval_path.pickle", "rb") as file:
  data = pickle.load(file)

complete_path = data["complete_path"]
weight = data["weight"]
print(weight)

209.0


In [17]:
m_1 = Map(
  basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
  center=cities[11]['coords'],
  zoom=10.5,
)

draw_lines(m_1, graph)
draw_cities(m_1, True)
draw_path(m_1, complete_path)

m_1

Map(center=[49.258352984690276, 4.030731630624372], controls=(ZoomControl(options=['position', 'zoom_in_text',…